### A case study: using CLUE datasets of Melbourne Open Data

# **Where should public transportation be enhanced in the city of Melbourne?**

The City of Melbourne has always evolved, and so have the numbers of its residential dwellings and employment population (based on data available on Melbourne Open Data). Without a consistent or parallel increase of off-street parking, the city will face the difficulty in providing enough spaces for parking of private transportation. To resolve this problem, the city planner needs to locate the potential areas where extension of public transportation should be implemented to replace the need for off-street parking.


## Purpose: 
This case study is intended to provide educational instructions of how CLUE datasets can be used for city planners or individuals whose have some knowledge in data analysis.

## Pre-requisite skills:
- Python scripting (Basic)
- Data analysis (Basic EDA and visualizations)

## Solution:
By looking at the population trend of residential dwellings and employments at different Melbourne city blocks, the city planner can project amount of public transportation services necessary to sufficiently accommodate potential travellers to or/and from those city blocks.



## Interactive Widgets

In [91]:
#Create a widget to select the range of study years
import ipywidgets as widgets
style = {'description_width': 'initial'} #for long label of the widgets
#Year-to-include slider
years = widgets.IntRangeSlider(value=[2002, 2020], min=2002, max=2020, step=1, description='Years to include:', style=style)
#Display option: impact level
show_impact = widgets.IntSlider(value=15, min=10, max=100, step=1, description='Impact index from:',style=style)
#Employment checkbox
chk_employment = widgets.Checkbox(value=True, description='Employment', indent=False)
#Residential Dwellings checkbox
chk_resident = widgets.Checkbox(value=True, description='Residential Dwellings', indent=False )


## Main function for interactivity

In [92]:
#Define function to execute when using interactive tools
def show_interaction(Year_to_include,Employment,Residential_dwellings, Impact_level):
    from IPython.display import display
    #Must import libraries and datasets for a self-contain function
    import pandas as pd
    import warnings
    warnings.filterwarnings("ignore") #suppress dataframe warning

    data = pd.read_csv('merged_data.csv') #the dataset that contains all the THREE features (employment, resident & parking)
    data.set_index(['Census year','Block ID'],inplace=True) #Turn the data to pandas series
    print('='*23)
    print('|DATA ANALYSIS RESULTS|')
    print('='*23)
    print('(NOTE: The city blocks without parking information available are ignored by our analyzing model.\n\n')
    if (not(Employment)) & (not(Residential_dwellings)):
        Employment = True  #Prevent both data features are unticked ==> Error
    #Get the filtered data for analysis
    X, Y = Get_data(data,Year_to_include,Employment,Residential_dwellings)
    #Build regression model
    lm = lm_model(X,Y)
    #Get the predicted results
    predict_results = Get_predict_result(data,lm, Employment, Residential_dwellings)
    predict_results = predict_results[predict_results['Impact_index']>=Impact_level] #only plot city block that have higher impact level

    #Plot the barchart of the predicted results
    bar_plot(predict_results, Impact_level);
    #Show table of the impacted blocks in detail table
    print('\n\n The table below shows the impacted blocks (limited parking) by the order of the impact index.\n')
    display(predict_results.sort_values('Impact_index',ascending = False))
    #Plot the folium choropleth map
    print('The impacted city blocks can be seen via the map here:')
    display(plot_map(predict_results))

## Demo Interactivity

In [93]:
#Display the interactive tools
from ipywidgets import interact_manual
interact_manual.opts['manual_name'] = 'Start Analyzing'
interact_manual(show_interaction, Year_to_include=years, Employment=chk_employment, Residential_dwellings = chk_resident, Impact_level = show_impact);

interactive(children=(IntRangeSlider(value=(2002, 2020), description='Years to include:', max=2020, min=2002, …

# **Working with Residential dwelling datasets**

In [95]:
#CLUE datasets
#======================================================
import pandas as pd
#Residential dwellings data
resident = pd.read_csv('Residential_dwellings.csv')
#Total records before cleaning
print('Total records before cleaning is',len(resident))
#Data cleaning (remove duplicated values)
resident.drop_duplicates(inplace = True)
#Total records after cleaning
print('Total records after cleaning is',len(resident))
#Select only necessary columns
selected_columns =['Census year','Block ID','CLUE small area','Dwelling number']
resident = resident[selected_columns]
#Sum the Dwelling number based on Block ID and Year
resident_by_block = resident.groupby(['Census year','Block ID']).agg({'CLUE small area':'max','Dwelling number':'sum'})
print('Total records after grouping (by Block ID) is',len(resident_by_block))
#Detail of the residential dwellings by block
resident_by_block.head()


# **Working with Emploment datasets**

In [96]:
#Employment data
employment = pd.read_csv('Employment_by_block_by_CLUE_industry.csv')
#Total records before cleaning
print('Total records before cleaning is',len(employment))
#Cleaning the data (remove missing values at [Total employment in block])
employment.dropna(subset=['Total employment in block'],inplace=True)
#Total records after cleaning
print('Total records afer cleaning is',len(employment))
#Select only necessary columns
selected_columns =['Census year','Block ID','Total employment in block']
employment = employment[selected_columns]
#Sum the employment based on Block ID and Year
employment_by_block = employment.groupby(['Census year','Block ID']).sum()
print('Total records after grouping (by Block ID) is',len(employment_by_block))
#Detail of the employment_by_block
employment_by_block.head()

# **Working with Off-street Parking datasets**

In [97]:
#Off-street parking data
parking = pd.read_csv('Off-street_car_parks_with_capacity_and_type.csv')
#Total records before cleaning
print('Total records before cleaning is',len(parking))
#Data cleaning (remove duplicate values)
parking.drop_duplicates(inplace=True)
#Total records after cleaning
print('Total records after cleaning is',len(parking))
#Select only necessary columns
selected_columns =['Census year','Block ID','Parking spaces']
parking = parking[selected_columns]
#Sum the parking based on Block ID and Year
parking_by_block = parking.groupby(['Census year','Block ID']).sum()
print('Total records after grouping (by Block ID) is',len(parking_by_block))
#Detail of the employment_by_block
parking_by_block.head()


# **Merge and explore the 3 datasets (Resident, Employment, & Parking)**

Formulate the model to identify a suitable number of parking slots based on residential dwelling or employment.

In [99]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_theme(color_codes=True)

#Combine the datasets (only take the blocks that match between three datasets)
data = resident_by_block.merge(employment_by_block, left_index= True, right_index=True)
data = data.merge(parking_by_block,left_index=True,right_index=True)
print(data.head())
#Prepare data for scatter plots
x1 = data['Total employment in block']
x2 = data['Dwelling number']
y = data['Parking spaces']
#Plot the data points (Parking Spaces vs. Total Employment) and regression line
plt.plot(x1,y,'x')
a, b = np.polyfit(x1,y,1)
plt.plot(x1, a*x1 + b)
plt.title('Parking Spaces vs. Total Employment')
plt.ylabel('Parking Spaces')
plt.xlabel('Total Employment')
plt.show()
#Plot the data points (Parking Spaces vs. Dwelling Number) and regression line
plt.plot(x2,y,'+')
a, b = np.polyfit(x2,y,1)
plt.plot(x2, a*x2 + b)
plt.title('Parking Spaces vs. Dwelling Number')
plt.ylabel('Parking Spaces')
plt.xlabel('Dwelling Number')
plt.show()

#NOTE: This dataset here ('data') need to be exported to GitHub and named as "merged_data.csv" to allow interactive (function) to work

# **Modeling and Analyzing**

## Prepare the filtered data records

In [101]:
#Prepare X and Y data for model training
def Get_data(Data, Year_to_include,Employment,Resident_dwellings):
  import numpy as np
  features = np.array(['Total employment in block', 'Dwelling number'])
  features_to_include = features[[Employment,Resident_dwellings]] #Define the data features for the study
  data_to_include = Data.loc[min(Year_to_include):max(Year_to_include)] #Get the wanted rows
  X = np.array(data_to_include[features_to_include]) #Get the independent variable(s) based on the selected features
  Y = np.array(data_to_include['Parking spaces']) #Get the dependent variable
  return X, Y
#print('The study period is between: %d and %d'%(min(years_to_include),max(years_to_include)))


## Build linear regression model

In [102]:
def lm_model(X, Y):
  import sklearn
  from sklearn import linear_model
  from sklearn.ensemble import IsolationForest
  iso = IsolationForest(contamination=0.1)#use outliers detector
  yhat = iso.fit_predict(X) #Search for outliers
  lm = linear_model.LinearRegression() #linear regression model
  lm.fit(X[yhat!=-1],Y[yhat!=-1]) # Train the model excluding outliers
  return lm

## Predict the parking problem based on the latest year (2020)

In [103]:
#Predict parking issue in 2020
def Get_predict_result(Data, Model, Employment, Residential_dwellings):
  import numpy as np
  data_latest = Data.loc[years.max] # Get data of 2020
  features = np.array(['Total employment in block', 'Dwelling number'])
  features_to_include = features[[Employment,Residential_dwellings]] #Define the data features for the study
  result = np.round(Model.predict(np.array(data_latest[features_to_include]))) #Predict based on the selected features
  data_latest['Predict_parking'] = result.astype(int) #Add result to the data
  impact = result - data_latest['Parking spaces']
  impact[impact>=0] = 100*(impact[impact>=0]/(max(impact[impact>=0]))) # Normalize the impact index (0-100)
  data_latest['Impact_index'] = impact.astype(int) #Add impact index to the data
  return data_latest

## Plot barchart of the predict results

In [104]:
#visualize the impacted city blocks
def bar_plot(Data, Impact_level):
  import matplotlib.pyplot as plt
  import numpy as np
  import seaborn as sns
  sns.set_theme(color_codes=True)
  #Only visualize the blocks greater than the impact level
  plot_data = Data['Impact_index']#Get only the data records with higher impact level
  plt.figure(figsize=(13,len(plot_data)/3))
  plt.xlabel('Impact Index (Higher index indicates serious shortage of off-street parking)')
  plt.ylabel('Block ID of Melbourne city')
  plt.title('Impacted Blocks vs. Impact Index\n Shows impact index from '+str(Impact_level)+' to 100')
  plt.barh (plot_data.index.astype(str),plot_data.values)
  plt.show()

# Plot the map using Folium Choropleth

In [105]:
def plot_map(clue_data):
  import json
  import folium
  #Load the csv and json data for map plotting
  json_file = open('clue_blocks.geojson') 
  clue_geo = json.load(json_file)

  clue_data.reset_index(inplace=True) #turn the index (Block_ID) into a column in the dataframe (clue_data)
  clue_data['Block ID'] = clue_data['Block ID'].astype(str) # conver [Block ID] to str, so that it is compatible with json content

  #Create the initial map
  fmap = folium.Map(location=[-37.811600, 144.964610],
            tiles = 'Stamen Toner',
            width = '70%',
            height = '100%',
            zoom_start=13)

  #create the choropleth layer and add to the map above
  choropleth = folium.Choropleth(
      geo_data=clue_geo,
      name='choropleth',
      data=clue_data,
      columns=['Block ID','Impact_index'],
      key_on='feature.properties.block_id',
      fill_color='YlOrRd',
      fill_opacity=1,
      line_opacity=0.5,
      nan_fill_color='cloud',
      nan_fill_opacity = 0.2,
      highlight=True,
      legend_name='Impact index'
  ).add_to(fmap)

  #Add more layers and tooltips
  choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['block_id','clue_area'],labels=True))
  return fmap